# Day 8: Working with APIs

## **Learning Objectives**
By the end of this lesson, students will be able to:
- Understand what APIs are and how they work
- Fetch data from APIs using the Fetch API
- Use useEffect to make API calls
- Handle loading states while data is being fetched
- Handle errors when API calls fail
- Display fetched data in components

---

## **Part 1: What is an API?**

### **API (Application Programming Interface)**

An API is a way for applications to communicate with each other. Think of it as a waiter in a restaurant:
- **You (Frontend)** = Customer who orders food
- **API** = Waiter who takes your order and brings food back
- **Server/Database** = Kitchen that prepares the food

### **REST API**

Most web APIs are REST APIs. They use HTTP methods:
- **GET** - Retrieve data (like reading)
- **POST** - Send new data (like creating)
- **PUT** - Update existing data
- **DELETE** - Remove data

### **API Example**

```
API URL: https://jsonplaceholder.typicode.com/users
Response: A list of user objects in JSON format

[
  { "id": 1, "name": "Leanne Graham", "email": "leanne@example.com" },
  { "id": 2, "name": "Ervin Howell", "email": "ervin@example.com" }
]
```

---


## **Part 2: The Fetch API**

### **Basic Fetch with Async/Await**

```javascript
async function fetchData() {
  try {
    const response = await fetch('https://api.example.com/data');
    const data = await response.json();
    console.log(data);
  } catch (error) {
    console.error('Error:', error);
  }
}
```

### **Step-by-Step Breakdown**

```javascript
async function fetchUsers() {
  try {
    // 1. Make the request and wait for response
    const response = await fetch('https://jsonplaceholder.typicode.com/users');
    console.log('Response received:', response);
    
    // 2. Convert response to JSON and wait
    const data = await response.json();
    console.log('Data:', data);
    
    // 3. Use the data
    return data;
  } catch (error) {
    // 4. Handle any errors
    console.error('Error:', error);
  }
}
```

### **Checking Response Status**

```javascript
async function fetchData() {
  try {
    const response = await fetch('https://api.example.com/data');
    
    // Check if request was successful
    if (!response.ok) {
      throw new Error(`HTTP error! Status: ${response.status}`);
    }
    
    const data = await response.json();
    return data;
  } catch (error) {
    console.error('Error:', error);
  }
}
```

---


## **Part 3: Fetching Data with useEffect**

### **Basic Example with Async/Await**

```javascript
import { useState, useEffect } from 'react';

function UserList() {
  const [users, setUsers] = useState([]);
  
  useEffect(() => {
    async function fetchUsers() {
      try {
        const response = await fetch('https://jsonplaceholder.typicode.com/users');
        const data = await response.json();
        setUsers(data);
      } catch (error) {
        console.error('Error:', error);
      }
    }
    
    fetchUsers();
  }, []); // Empty array = fetch once on mount
  
  return (
    <div>
      <h2>Users</h2>
      {users.map(user => (
        <p key={user.id}>{user.username}</p>
      ))}
    </div>
  );
}
```

**Important Notes:**
- We create an async function INSIDE useEffect
- We call that function immediately
- We can't make useEffect itself async: ❌ `useEffect(async () => {...})`
- Empty `[]` ensures it runs only on mount
- Without it: infinite loop (fetch → update state → re-render → fetch → ...)

---


## **Part 4: Loading States**

### **Why Loading States?**

Users need to know data is being fetched. Always show a loading indicator.

```javascript
import { useState, useEffect } from 'react';

function UserList() {
  const [users, setUsers] = useState([]);
  const [loading, setLoading] = useState(true);
  
  useEffect(() => {
    async function fetchUsers() {
      try {
        const response = await fetch('https://jsonplaceholder.typicode.com/users');
        const data = await response.json();
        setUsers(data);
        setLoading(false); // Done loading
      } catch (error) {
        console.error('Error:', error);
        setLoading(false);
      }
    }
    
    fetchUsers();
  }, []);
  
  if (loading) {
    return <p>Loading users...</p>;
  }
  
  return (
    <div>
      <h2>Users</h2>
      {users.map(user => (
        <p key={user.id}>{user.name}</p>
      ))}
    </div>
  );
}
```

---


## **Part 5: Error Handling**

### **Proper Error Handling with Try/Catch**

```javascript
import { useState, useEffect } from 'react';

function UserList() {
  const [users, setUsers] = useState([]);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);
  
  useEffect(() => {
    async function fetchUsers() {
      try {
        const response = await fetch('https://jsonplaceholder.typicode.com/users');
        
        if (!response.ok) {
          throw new Error('Failed to fetch users');
        }
        
        const data = await response.json();
        setUsers(data);
      } catch (error) {
        setError(error.message);
      } finally {
        setLoading(false); // Always runs, whether success or error
      }
    }
    
    fetchUsers();
  }, []);
  
  if (loading) return <p>Loading...</p>;
  if (error) return <p>Error: {error}</p>;
  
  return (
    <div>
      <h2>Users ({users.length})</h2>
      {users.map(user => (
        <p key={user.id}>{user.name}</p>
      ))}
    </div>
  );
}
```

**Benefits of try/catch/finally:**
- `try` - Code that might fail
- `catch` - Handle the error
- `finally` - Always runs (perfect for setLoading(false))

---


## **Part 6: Best Practices Pattern**

### **The Complete Pattern**

This is the standard pattern you should use for all API calls:

```javascript
import { useState, useEffect } from 'react';

function DataComponent() {
  const [data, setData] = useState([]);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);
  
  useEffect(() => {
    async function fetchData() {
      try {
        const response = await fetch('API_URL_HERE');
        
        if (!response.ok) {
          throw new Error(`HTTP error! Status: ${response.status}`);
        }
        
        const result = await response.json();
        setData(result);
      } catch (error) {
        setError(error.message);
      } finally {
        setLoading(false);
      }
    }
    
    fetchData();
  }, []);
  
  if (loading) return <div>Loading...</div>;
  if (error) return <div>Error: {error}</div>;
  
  return (
    <div>
      {/* Display your data here */}
    </div>
  );
}
```

**This pattern handles:**
✅ Loading state  
✅ Error state  
✅ Success state  
✅ Proper error checking  
✅ Clean async/await syntax  

---


## **Part 7: Re-fetching Data**

### **Fetch When Dependency Changes**

```javascript
import { useState, useEffect } from 'react';

function UserPosts() {
  const [userId, setUserId] = useState(1);
  const [posts, setPosts] = useState([]);
  const [loading, setLoading] = useState(true);
  
  useEffect(() => {
    async function fetchPosts() {
      setLoading(true);
      try {
        const response = await fetch(
          `https://jsonplaceholder.typicode.com/posts?userId=${userId}`
        );
        const data = await response.json();
        setPosts(data);
      } catch (error) {
        console.error('Error:', error);
      } finally {
        setLoading(false);
      }
    }
    
    fetchPosts();
  }, [userId]); // Re-fetch when userId changes
  
  return (
    <div>
      <button onClick={() => setUserId(userId - 1)} disabled={userId === 1}>
        Previous User
      </button>
      <button onClick={() => setUserId(userId + 1)}>
        Next User
      </button>
      
      <h2>Posts by User {userId}</h2>
      {loading ? (
        <p>Loading...</p>
      ) : (
        posts.map(post => (
          <div key={post.id}>
            <h3>{post.title}</h3>
            <p>{post.body}</p>
          </div>
        ))
      )}
    </div>
  );
}
```

---


## **Part 8: Complete Example - Nigerian News App**

Let's build a news app that fetches articles from an API.

### **NewsApp.jsx**

```javascript
import { useState, useEffect } from 'react';
import './NewsApp.css';

function NewsApp() {
  const [articles, setArticles] = useState([]);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);
  const [category, setCategory] = useState('all');
  
  useEffect(() => {
    async function fetchNews() {
      setLoading(true);
      setError(null);
      
      try {
        // Using JSONPlaceholder as demo API
        const response = await fetch(
          'https://jsonplaceholder.typicode.com/posts?_limit=12'
        );
        
        if (!response.ok) {
          throw new Error('Failed to fetch news');
        }
        
        const data = await response.json();
        
        // Transform data to look like news articles
        const newsArticles = data.map(post => ({
          id: post.id,
          title: post.title,
          body: post.body,
          category: ['Politics', 'Business', 'Sports', 'Entertainment'][
            Math.floor(Math.random() * 4)
          ],
          date: new Date().toLocaleDateString('en-NG'),
          author: `Author ${post.userId}`
        }));
        
        setArticles(newsArticles);
      } catch (err) {
        setError(err.message);
      } finally {
        setLoading(false);
      }
    }
    
    fetchNews();
  }, []);
  
  // Filter articles by category
  const filteredArticles = category === 'all' 
    ? articles 
    : articles.filter(article => article.category === category);
  
  return (
    <div className="news-app">
      <header className="news-header">
        <h1>🇳🇬 Nigerian News</h1>
        <p>Stay updated with the latest news</p>
      </header>
      
      {/* Category Filter */}
      <div className="category-filter">
        {['all', 'Politics', 'Business', 'Sports', 'Entertainment'].map(cat => (
          <button 
            key={cat}
            className={category === cat ? 'active' : ''}
            onClick={() => setCategory(cat)}
          >
            {cat === 'all' ? 'All' : cat}
          </button>
        ))}
      </div>
      
      {/* Loading State */}
      {loading && (
        <div className="loading">
          <div className="spinner"></div>
          <p>Loading news...</p>
        </div>
      )}
      
      {/* Error State */}
      {error && (
        <div className="error">
          <p>❌ {error}</p>
          <button onClick={() => window.location.reload()}>
            Try Again
          </button>
        </div>
      )}
      
      {/* Articles */}
      {!loading && !error && (
        <div className="articles">
          {filteredArticles.length === 0 ? (
            <p className="no-results">No articles found in this category.</p>
          ) : (
            filteredArticles.map(article => (
              <div key={article.id} className="article-card">
                <span className="category-badge">{article.category}</span>
                <h2>{article.title}</h2>
                <p className="article-body">{article.body}</p>
                <div className="article-footer">
                  <span>By {article.author}</span>
                  <span>{article.date}</span>
                </div>
              </div>
            ))
          )}
        </div>
      )}
    </div>
  );
}

export default NewsApp;
```

### **NewsApp.css**

```css
.news-app {
  max-width: 1200px;
  margin: 0 auto;
  padding: 20px;
}

.news-header {
  text-align: center;
  margin-bottom: 30px;
  color: #008751;
}

.news-header h1 {
  font-size: 2.5em;
  margin-bottom: 10px;
}

.category-filter {
  display: flex;
  gap: 10px;
  margin-bottom: 30px;
  justify-content: center;
  flex-wrap: wrap;
}

.category-filter button {
  padding: 10px 20px;
  border: 2px solid #008751;
  background: white;
  color: #008751;
  border-radius: 25px;
  cursor: pointer;
  transition: all 0.3s;
}

.category-filter button.active {
  background: #008751;
  color: white;
}

.category-filter button:hover {
  transform: translateY(-2px);
}

.loading {
  text-align: center;
  padding: 60px;
}

.spinner {
  width: 50px;
  height: 50px;
  border: 5px solid #f3f3f3;
  border-top: 5px solid #008751;
  border-radius: 50%;
  margin: 0 auto 20px;
  animation: spin 1s linear infinite;
}

@keyframes spin {
  0% { transform: rotate(0deg); }
  100% { transform: rotate(360deg); }
}

.error {
  text-align: center;
  padding: 60px;
  color: #d32f2f;
}

.error button {
  margin-top: 20px;
  padding: 10px 20px;
  background: #d32f2f;
  color: white;
  border: none;
  border-radius: 5px;
  cursor: pointer;
}

.articles {
  display: grid;
  grid-template-columns: repeat(auto-fill, minmax(350px, 1fr));
  gap: 20px;
}

.article-card {
  background: white;
  padding: 20px;
  border-radius: 10px;
  box-shadow: 0 2px 8px rgba(0,0,0,0.1);
  transition: transform 0.3s;
}

.article-card:hover {
  transform: translateY(-5px);
  box-shadow: 0 4px 16px rgba(0,0,0,0.2);
}

.category-badge {
  display: inline-block;
  background: #008751;
  color: white;
  padding: 5px 12px;
  border-radius: 15px;
  font-size: 12px;
  margin-bottom: 10px;
}

.article-card h2 {
  color: #333;
  margin-bottom: 15px;
  font-size: 1.3em;
}

.article-body {
  color: #666;
  line-height: 1.6;
  margin-bottom: 15px;
}

.article-footer {
  display: flex;
  justify-content: space-between;
  color: #999;
  font-size: 0.9em;
  padding-top: 15px;
  border-top: 1px solid #eee;
}

.no-results {
  text-align: center;
  color: #999;
  padding: 40px;
}
```

### **What We Used:**
✅ **Fetch API** with async/await  
✅ **Loading state** with spinner animation  
✅ **Error handling** with try/catch  
✅ **Conditional rendering** for loading/error/success states  
✅ **Data transformation** to format API response  
✅ **Filtering** based on user selection  

---


## **Part 9: Tasks**

## 🎯 Task 1: Random User Generator

**Task:** Fetch and display random users from an API

**Requirements:**
1. Create `RandomUser.jsx`
2. Fetch from: `https://randomuser.me/api/`
3. Display: photo, name, email, phone, location
4. Show loading state
5. Add "Get New User" button to fetch another
6. Handle errors properly

**Sample Output:**
```
👤 Random User Profile

[Loading...]

or

[Profile Photo]
John Doe
john@example.com
08012345678
Lagos, Nigeria

[Get New User]
```

**Challenge:** Add a counter showing total users viewed

---

## 🎯 Task 2: Nigerian Universities Directory

**Task:** Fetch and display Nigerian universities

**Requirements:**
1. Create `UniversityDirectory.jsx`
2. Fetch from: `http://universities.hipolabs.com/search?country=Nigeria`
3. Display: name, state/province, website (as clickable link)
4. Show total count
5. Add search input to filter results
6. Loading and error states

**Sample Output:**
```
🎓 Nigerian Universities

Search: [_________]

Loading...

or

Showing 150 universities

University of Lagos
Lagos State
🔗 www.unilag.edu.ng

University of Ibadan
Oyo State
🔗 www.ui.edu.ng
```

**Challenge:** Sort alphabetically

---

## 🎯 Task 3: Quote of the Day

**Task:** Fetch inspirational quotes

**Requirements:**
1. Create `QuoteGenerator.jsx`
2. Fetch from: `https://zenquotes.io/api/random`
3. Display:
   - The **quote text** (`q`)  
   - The **author** (`a`)  
   - The **quote length** — *(calculated using `quote.q.length`)* 
4. "New Quote" button
5. Loading state between quotes
6. Error handling
7. Track how many quotes viewed

**Sample Output:**
```
💬 Daily Inspiration

"The only way to do great work is to love what you do."
- Steve Jobs

Length: 52 characters
Quotes viewed: 5

[New Quote]
```

**Challenge:** Add "Copy Quote" button

---

## 🎯 Task 4: GitHub User Search

**Task:** Search GitHub users by username

**Requirements:**
1. Create `GitHubSearch.jsx`
2. Input field for username
3. Fetch from: `https://api.github.com/users/{username}`
4. Display: avatar, username, name, bio, public repos count, followers
5. Show "Search" button
6. Loading state while fetching
7. Error message if user not found

**Sample Output:**
```
🔍 GitHub User Search

Username: [octocat_____] [Search]

Loading...

or

[Avatar]
@octocat
The Octocat
How people build software.

📚 Repos: 8
👥 Followers: 5000
```

**Challenge:** Show list of repositories

---


## **Part 10: Review**

### **Key Takeaways**
✅ APIs allow communication between applications  
✅ Use `fetch()` to get data from APIs  
✅ Always use useEffect with empty `[]` for API calls on mount  
✅ Handle three states: loading, error, success  
✅ Use async/await for cleaner code  
✅ Check `response.ok` before parsing JSON  
✅ Re-fetch data when dependencies change  

### **Common Mistakes**
- Not handling loading states (users see nothing)
- Not handling errors (app breaks silently)
- Forgetting empty dependency array (infinite loop)
- Not checking `response.ok` (errors not caught)
- Not using try/catch with async/await
- Fetching data outside useEffect

### **Best Practices**
- Always show loading indicators
- Display user-friendly error messages
- Use async/await for readability
- Keep API logic in useEffect
- Extract fetch logic into separate functions for reusability
- Handle edge cases (empty data, network issues)

---

## **Next Lesson Preview**
Tomorrow we'll learn about **Forms & Validation** - building forms with controlled inputs and validating user data!